In [21]:
import requests

In [22]:
req_ses = requests.session()
headers = {}
headers['Accept-Encoding'] = "gzip, deflate, br"
headers['Accept-Language'] = "zh-CN,zh;q=0.9"

In [35]:
url = 'https://api.bilibili.com/x/v2/dm/web/seg.so?type=1&oid=454999537&pid=464724213&segment_index=2'
html = req_ses.get(url=url, headers=headers)

In [36]:
data_raw = html.content
data_raw

b'\nZ\x08\x80\xec\xda\xcf\xcc\xdc\xf6g\x10\x9d\xaf"\x18\x01 \x19(\xff\xff\xff\x072\x0882d36db2:\x0f\xe6\x88\x91\xe6\x80\x9d\xe6\x95\x85\xe6\x88\x91\xe5\x9c\xa8@\xa4\x9f\xb1\x8d\x06H\nb\x1158505996354893312p\xd4\xb2\xcc\x85\x01\xa2\x01\x010\xaa\x01\x010\nU\x08\x80\xcc\xcf\x8e\x8f\xdd\xf6g\x10\x99\xfe\'\x18\x01 \x19(\xff\xff\xff\x072\x086cb166af:\t\xe5\xa5\xbd\xe6\xb5\xaa\xe6\xbc\xab@\xc6\x9f\xb1\x8d\x06H\nb\x1158506014203569664p\xa7\xe8\xef\x04x\x05\xa2\x01\x010\xaa\x01\x010\nk\x08\x80\xbc\xd7\x81\xc1\x9f\xf8g\x10\xc6\xf9!\x18\x05 \x19(\xff\xff\xff\x072\x08703bfcfc:!\xe9\x87\x8f\xe5\xad\x90\xe5\x9c\xa8\xe8\xa2\xab\xe8\xa7\x82\xe6\xb5\x8b\xe6\x97\xb6\xe4\xb8\x8d\xe4\xbc\x9a\xe8\xa1\xb0\xe5\x8f\x98@\x89\x83\xb2\x8d\x06H\nb\x1158512693387451904p\x91\x9c\xd8-\xa2\x01\x010\xaa\x01\x010\n\x9d\x01\x08\x80\xa4\xac\xc6\x98\xab\xf8g\x10\x94\xc5$\x18\x05 \x19(\xfb\xf4\xa0\x032\x0817832c65:Q\xe8\xbf\x99\xe8\xbe\xb9\xe6\x9c\x89\xe5\xbd\xa9\xe8\x9b\x8b\xe7\xbb\x93\xe5\xb1\x80\xef\xbc\x8c\xe5\x9c\xa8\

In [37]:
def read_varint(stream: bytes):
    """
    读取 varint。
    Args:
        stream (bytes): 字节流。
    Returns:
        tuple[int, int]，真实值和占用长度。
    """
    value = 0
    position = 0
    shift = 0
    while True:
        if position >= len(stream):
            break
        byte = stream[position]
        value += (byte & 0b01111111) << shift
        if byte & 0b10000000 == 0:
            break
        position += 1
        shift += 7
    return value, position + 1


class BytesReader:
    """
    读字节流助手类。
    """
    def __init__(self, stream: bytes):
        """
        Args:
            stream (bytes): 字节流
        """
        self.__stream = stream
        self.__offset = 0

    def has_end(self):
        """
        是否已读到末尾
        Returns:
            bool。
        """
        return self.__offset >= len(self.__stream)

    def double(self, LE=False):
        """
        读 double。
        Args:
            LE (bool): 为小端。
        Returns:
            float。
        """
        data = struct.unpack("<d" if LE else ">d", self.__stream[self.__offset:self.__offset + 8])
        self.__offset += 8
        return data[0]

    def float(self, LE=False):
        """
        读 float。
        Args:
            LE (bool): 为小端。
        Returns:
            float。
        """
        stream = self.__stream[self.__offset:self.__offset + 4]
        data = struct.unpack("<f" if LE else ">f", stream)
        self.__offset += 4
        return data[0]

    def varint(self):
        """
        读 varint。
        Returns:
            int。
        """
        d, l = read_varint(self.__stream[self.__offset:])
        self.__offset += l
        return d

    def byte(self):
        """
        读 byte。
        Returns：
            int。
        """
        data = self.__stream[self.__offset]
        self.__offset += 1
        return data

    def string(self, encoding="utf8"):
        """
        读 string。
        Args:
            encoding (str):  编码方式。
        Returns:
            str。
        """
        str_len = self.varint()
        data = self.__stream[self.__offset:self.__offset + str_len]
        self.__offset += str_len
        return data.decode(encoding=encoding, errors="ignore")

    def bool(self):
        """
        读 bool。
        Returns:
            bool。
        """
        data = self.__stream[self.__offset]
        self.__offset += 1
        return data == 1

    def bytes_string(self):
        """
        读原始字节流。
        Returns:
            bytes。
        """
        str_len = self.varint()
        data = self.__stream[self.__offset:self.__offset + str_len]
        self.__offset += str_len
        return data

    def fixed16(self, LE=False):
        """
        读 Fixed int16。
        Args:
            LE (bool): 为小端。
        Returns:
            int。
        """
        data = struct.unpack("<h" if LE else ">h", self.__stream[self.__offset:self.__offset + 2])
        self.__offset += 2
        return data[0]

    def fixed32(self, LE=False):
        """
        读 Fixed int32.
        Args:
            LE (bool): 为小端。
        Returns:
            int。
        """
        data = struct.unpack("<i" if LE else ">i", self.__stream[self.__offset:self.__offset + 4])
        self.__offset += 4
        return data[0]

    def fixed64(self, LE=False):
        """
        读 Fixed int64。
        Args:
            LE (bool): 为小端。
        Returns:
            int。
        """
        data = struct.unpack("<q" if LE else ">q", self.__stream[self.__offset:self.__offset + 8])
        self.__offset += 8
        return data[0]

    def ufixed16(self, LE=False):
        """
        读 Unsigned fixed Int16。
        Args:
            LE (bool): 为小端。
        Returns:
            int。
        """
        data = struct.unpack("<H" if LE else ">H", self.__stream[self.__offset:self.__offset + 2])
        self.__offset += 2
        return data[0]

    def ufixed32(self, LE=False):
        """
        读 Unsigned fixed Int32。
        Args:
            LE (bool): 为小端。
        Returns:
            int。
        """
        data = struct.unpack("<I" if LE else ">I", self.__stream[self.__offset:self.__offset + 4])
        self.__offset += 4
        return data[0]

    def ufixed64(self, LE=False):
        """
        读 Unsigned fixed Int64。
        Args:
            LE (bool): 为小端。
        Returns:
            int。
        """
        data = struct.unpack("<Q" if LE else ">Q", self.__stream[self.__offset:self.__offset + 8])
        self.__offset += 8
        return data[0]

    def set_pos(self, pos: int):
        """
        设置读取起始位置。
        Args:
            pos (int): 读取起始位置。
        """
        if pos < 0:
            raise Exception("读取位置不能小于 0")

        if pos >= len(self.__stream):
            raise Exception("读取位置超过字节流长度")

        self.__offset = pos

    def get_pos(self):
        """
        获取当前位置。
        Returns:
            int, 当前位置。
        """
        return self.__offset

In [38]:
"""
引用自https://github.com/MoyuScript/bilibili-api/blob/a40d743089b70a0d0c73207690eb33a38cf28804/bilibili_api/utils/utils.py
"""
def crack_uid(crc32: str):

    __CRCPOLYNOMIAL = 0xEDB88320
    __crctable = [None] * 256
    __index = [None] * 4

    def __create_table():
        for i in range(256):
            crcreg = i
            for j in range(8):
                if (crcreg & 1) != 0:
                    crcreg = __CRCPOLYNOMIAL ^ (crcreg >> 1)
                else:
                    crcreg >>= 1
            __crctable[i] = crcreg

    __create_table()

    def __crc32(input_):
        if type(input_) != str:
            input_ = str(input_)
        crcstart = 0xFFFFFFFF
        len_ = len(input_)
        for i in range(len_):
            index = (crcstart ^ ord(input_[i])) & 0xFF
            crcstart = (crcstart >> 8) ^ __crctable[index]
        return crcstart

    def __crc32lastindex(input_):
        if type(input_) != str:
            input_ = str(input_)
        crcstart = 0xFFFFFFFF
        len_ = len(input_)
        index = None
        for i in range(len_):
            index = (crcstart ^ ord(input_[i])) & 0xFF
            crcstart = (crcstart >> 8) ^ __crctable[index]
        return index

    def __getcrcindex(t):
        for i in range(256):
            if __crctable[i] >> 24 == t:
                return i
        return -1

    def __deepCheck(i, index):
        tc = 0x00
        str_ = ""
        hash_ = __crc32(i)
        tc = hash_ & 0xFF ^ index[2]
        if not (57 >= tc >= 48):
            return [0]
        str_ += str(tc - 48)
        hash_ = __crctable[index[2]] ^ (hash_ >> 8)

        tc = hash_ & 0xFF ^ index[1]
        if not (57 >= tc >= 48):
            return [0]
        str_ += str(tc - 48)
        hash_ = __crctable[index[1]] ^ (hash_ >> 8)

        tc = hash_ & 0xFF ^ index[0]
        if not (57 >= tc >= 48):
            return [0]
        str_ += str(tc - 48)
        hash_ = __crctable[index[0]] ^ (hash_ >> 8)

        return [1, str_]

    ht = int(crc32, 16) ^ 0xFFFFFFFF
    i = 3
    while i >= 0:
        __index[3-i] = __getcrcindex(ht >> (i*8))
        # pylint: disable=invalid-sequence-index
        snum = __crctable[__index[3-i]]
        ht ^= snum >> ((3-i)*8)
        i -= 1
    for i in range(10000000):
        lastindex = __crc32lastindex(i)
        if lastindex == __index[3]:
            deepCheckData = __deepCheck(i, __index)
            if deepCheckData[0]:
                break
    if i == 10000000:
        return -1
    return str(i) + deepCheckData[1]

In [39]:
import time
from enum import Enum

class FontSize(Enum):
    """
    字体大小枚举。
    """
    EXTREME_SMALL = 12
    SUPER_SMALL = 16
    SMALL = 18
    NORMAL = 25
    BIG = 36
    SUPER_BIG = 45
    EXTREME_BIG = 64


class Mode(Enum):
    """
    弹幕模式枚举。
    """
    FLY = 1
    TOP = 5
    BOTTOM = 4
    REVERSE = 6

class Danmaku:
    """
    弹幕类。
    """
    def __init__(self,
                 text: str,
                 dm_time: float = 0.0,
                 send_time: float = time.time(),
                 crc32_id: str = None,
                 color: str = 'ffffff',
                 weight: int = -1,
                 id_: int = -1,
                 id_str: str = "",
                 action: str = "",
                 mode: Mode = Mode.FLY,
                 font_size: FontSize = FontSize.NORMAL,
                 is_sub: bool = False,
                 pool: int = -1,
                 attr: int = -1):
        """
        Args:
            text      (str)               : 弹幕文本。
            dm_time   (float, optional)   : 弹幕在视频中的位置，单位为秒。Defaults to 0.0.
            send_time (float, optional)   : 弹幕发送的时间。Defaults to time.time().
            crc32_id  (str, optional)     : 弹幕发送者 UID 经 CRC32 算法取摘要后的值。Defaults to None.
            color     (str, optional)     : 弹幕十六进制颜色。Defaults to "ffffff".
            weight    (int, optional)     : 弹幕在弹幕列表显示的权重。Defaults to -1.
            id_       (int, optional)     : 弹幕 ID。Defaults to -1.
            id_str    (str, optional)     : 弹幕字符串 ID。Defaults to "".
            action    (str, optional)     : 暂不清楚。Defaults to "".
            mode      (Mode, optional)    : 弹幕模式。Defaults to Mode.FLY.
            font_size (FontSize, optional): 弹幕字体大小。Defaults to FontSize.NORMAL.
            is_sub    (bool, optional)    : 是否为字幕弹幕。Defaults to False.
            pool      (int, optional)     : 暂不清楚。Defaults to -1.
            attr      (int, optional)     : 暂不清楚。 Defaults to -1.
        """
        self.text = text
        self.dm_time = dm_time
        self.send_time = send_time
        self.crc32_id = crc32_id
        self.color = color
        self.weight = weight
        self.id = id_
        self.id_str = id_str
        self.action = action
        self.mode = mode
        self.font_size = font_size
        self.is_sub = is_sub
        self.pool = pool
        self.attr = attr

        self.uid = None

    def __str__(self):
        ret = "uid: {uid}, 弹幕时间: {dm_time}, 弹幕文本: {text}".format(uid=self.uid, send_time=self.send_time, dm_time=self.dm_time, text=self.text)
        return ret

    def __len__(self):
        return len(self.text)

    def crack_uid(self):
        """
        暴力破解 UID，可能存在误差，请慎重使用。
        Returns:
            int: 真实 UID。
        """
#         self.uid = int(crack_uid(self.crc32_id))
        self.uid = None
        return self.uid

In [42]:
reader = BytesReader(data_raw)
while not reader.has_end():
    reader.varint() >> 3
    dm = Danmaku('')
    dm_pack_data = reader.bytes_string()
    dm_reader = BytesReader(dm_pack_data)

    while not dm_reader.has_end():
        data_type = dm_reader.varint() >> 3

        if data_type == 1:
            dm.id = dm_reader.varint()
        elif data_type == 2:
            dm.dm_time = dm_reader.varint() / 1000
        elif data_type == 3:
            dm.mode = dm_reader.varint()
        elif data_type == 4:
            dm.font_size = dm_reader.varint()
        elif data_type == 5:
            dm.color = hex(dm_reader.varint())[2:]
        elif data_type == 6:
            dm.crc32_id = dm_reader.string()
            dm.crack_uid()
        elif data_type == 7:
            dm.text = dm_reader.string()
        elif data_type == 8:
            dm.send_time = dm_reader.varint()
        elif data_type == 9:
            dm.weight = dm_reader.varint()
        elif data_type == 10:
            dm.action = dm_reader.varint()
        elif data_type == 11:
            dm.pool = dm_reader.varint()
        elif data_type == 12:
            dm.id_str = dm_reader.string()
        elif data_type == 13:
            dm.attr = dm_reader.varint()
        else:
            break
    print(dm)

uid: None, 弹幕时间: 563.101, 弹幕文本: 我思故我在
uid: None, 弹幕时间: 655.129, 弹幕文本: 好浪漫
uid: None, 弹幕时间: 556.23, 弹幕文本: 量子在被观测时不会衰变
uid: None, 弹幕时间: 598.676, 弹幕文本: 这边有彩蛋结局，在黑洞还没进去前关上折越会导致时空破碎
uid: None, 弹幕时间: 388.763, 弹幕文本: 逛gai233
uid: None, 弹幕时间: 665.789, 弹幕文本: 很伟大也很美丽
uid: None, 弹幕时间: 549.019, 弹幕文本: 遇事不决，量子力学
uid: None, 弹幕时间: 704.279, 弹幕文本:  穷 举 法
uid: None, 弹幕时间: 392.713, 弹幕文本: 宇宙之眼
uid: None, 弹幕时间: 564.705, 弹幕文本: 前面我思故我在什么鬼。非要扯哲学的话也是贝克莱的“存在即被感知”
uid: None, 弹幕时间: 572.653, 弹幕文本: 未被感知，则不存在
uid: None, 弹幕时间: 567.805, 弹幕文本: 概率波
uid: None, 弹幕时间: 389.393, 弹幕文本: 哈哈，方言逛街
uid: None, 弹幕时间: 468.569, 弹幕文本: 哈哈，八戒这边可就惨喽
uid: None, 弹幕时间: 677.321, 弹幕文本: 在的在的
uid: None, 弹幕时间: 691.065, 弹幕文本: 所以说他们吃什么活这么久
uid: None, 弹幕时间: 672.222, 弹幕文本: 嗯……
uid: None, 弹幕时间: 682.899, 弹幕文本: 好好看好好看
uid: None, 弹幕时间: 587.939, 弹幕文本: 第六个位置，就是宇宙之眼
uid: None, 弹幕时间: 685.619, 弹幕文本: 挪麦人真的使用穷举法破解了
uid: None, 弹幕时间: 697.299, 弹幕文本: 这是好几代挪麦人共同努力的成果
uid: None, 弹幕时间: 680.659, 弹幕文本: 在
uid: None, 弹幕时间: 680.744, 弹幕文本: 在
uid: None, 弹幕时间: 692.658, 弹幕文本: 恩.